In [1]:
import altair as alt
import seaborn as sns
from altair import datum
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

regioni_df = get_processed_dataset('dpc_regioni')
today = regioni_df.groupby('regione').last().reset_index()

## Riepilogo per regione

### Casi per regione

In [2]:
from altair import datum

c = alt.Chart(regioni_df).mark_line().encode(
        x='data:T',
        color='regione',
        tooltip=['regione', 'data', 'totale_casi', 'deceduti', 'totale_casi_per_1M_pop', 'deceduti_per_1M_pop']
    ).properties(
        height=600,
        width=800
    ).transform_filter(
        (datum.totale_casi > 1000)
    ).interactive()
    
@interact(campo=['totale_casi', 'deceduti'], logarithm=True, per1Mpop=False)
def plot_province(campo, logarithm, per1Mpop):    
    y = campo
    if per1Mpop:
        y += '_per_1M_pop'
    if logarithm:
        return c.encode(y = alt.Y(f'{y}:Q', scale=alt.Scale(type='log')))
    else:
        return c.encode(y = alt.Y(f'{y}:Q'))

interactive(children=(Dropdown(description='campo', options=('totale_casi', 'deceduti'), value='totale_casi'),…

In [3]:
@interact(campo=['totale_casi', 'deceduti', 'tamponi'], normalize=True)
def plot(campo, normalize):
    stack = "normalize" if normalize else "zero"
    
    return alt.Chart(regioni_df).mark_area().encode(
        alt.Y(campo, stack=stack),#, scale=alt.Scale(type='log')),
        x='data:T',
        color=alt.Color('regione', scale=alt.Scale(scheme='category20')),
        tooltip=['regione', 'data', 'totale_casi', 'deceduti', 'tamponi']
    ).properties(
        height=600,
        width=800
    ).transform_filter(
        (datum.totale_casi > 0)
    ).interactive()

interactive(children=(Dropdown(description='campo', options=('totale_casi', 'deceduti', 'tamponi'), value='tot…

In [5]:
@interact(campo=['percentuale_positivi', 'percentuale_nuovi_positivi', 'incremento', 'nuovi_tamponi', 'nuovi_positivi', 'nuovi_deceduti', 'tamponi', 'terapia_intensiva', 'totale_positivi', 'letalita', 'totale_casi_per_1M_pop', 'deceduti_per_1M_pop'], giorni_indietro=(-10,0))
def plot(campo='nuovi_deceduti', giorni_indietro=0):
    df = regioni_df.groupby('regione').nth(giorni_indietro-1).reset_index()
    c = alt.Chart(df).mark_bar().encode(
        x=campo,
        y=alt.Y('regione:N', sort='-x'),
    ).properties(
        height=600,
        width=800
    ).transform_filter(
            (datum.percentuale_positivi > 0)
    ).properties(title=str(df.data.iloc[0])).interactive()
    return c

interactive(children=(Dropdown(description='campo', index=5, options=('percentuale_positivi', 'percentuale_nuo…